<a href="https://colab.research.google.com/github/PedroFerreiradaCosta/Lectures/blob/master/Computational_neuroscience_ML_%26_the_brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Computational Neuroscience

## Day 2: Machine Learning and the brain

### **Pedro Ferreira da Costa**

In this tutorial we will work with brain imaging data from T1 scans.

This tutorial is composed of three parts:
1. You will build an Unsupervised Learning method from scratch
2. You will explore and apply the machine learning library **scikit-learn**
3. You will explore the deep learning library **tensorflow**

## Part 1 - Exploring the data and clustering

### Install dependencies

In [1]:
# Import dependencies
import nibabel as nib # to load our .nii.gz files
import matplotlib.pyplot as plt # to plot images

from ipywidgets import interact # to create the sliders
import numpy as np # library for mathematical python
import skimage.measure # we use this library to downsample the brain data
from google.colab import output # to clear the output
import glob # to obtain the name of the files
import random # useful for initializations


### Load data. How many values are included in each sample?

In [ ]:
# Download brain data we are using
!wget -O brain_0.nii.gz https://www.dropbox.com/s/oy5w9z6ro4aj3gr/sample_0.nii.gz?dl=0
!wget -O brain_1.nii.gz https://www.dropbox.com/s/8w3z96r9uqec8un/sample_1.nii.gz?dl=0
!wget -O brain_2.nii.gz https://www.dropbox.com/s/j8tqkbq0tk9kw3g/sample_2.nii.gz?dl=0
!wget -O brain_3.nii.gz https://www.dropbox.com/s/wt0i12tvnwyerto/sample_3.nii.gz?dl=0
!wget -O brain_4.nii.gz https://www.dropbox.com/s/gftadh8wrhx35e0/sample_4.nii.gz?dl=0
!wget -O brain_5.nii.gz https://www.dropbox.com/s/ldtu2kbjvxwbkxb/sample_5.nii.gz?dl=0
!wget -O brain_6.nii.gz https://www.dropbox.com/s/bcqpu3sgw9ci2no/sample_6.nii.gz?dl=0
!wget -O brain_7.nii.gz https://www.dropbox.com/s/3oktkjfykgcekjt/sample_7.nii.gz?dl=0
!wget -O brain_8.nii.gz https://www.dropbox.com/s/vquuqejlcryitee/sample_8.nii.gz?dl=0
!wget -O brain_9.nii.gz https://www.dropbox.com/s/v1f7j12qb7ae4rs/sample_9.nii.gz?dl=0
!wget -O brain_10.nii.gz https://www.dropbox.com/s/ky70636u3dmbule/sample_10.nii.gz?dl=0
!wget -O brain.nii.gz https://www.dropbox.com/s/9wfugvhuk1y9wq3/sample.nii.gz?dl=0

#clear output so we can see the next print
output.clear()

# Load brain data
brain  = nib.load('brain.nii.gz').get_fdata()

print(f"Each brain scan is an array of: {brain.shape}.")
print(f"Or {brain.shape[0]*brain.shape[1]*brain.shape[2]} voxels.")


### Let's visualize our brain data. Is there anything odd you see with out data?

In [ ]:
# Function that plots the brain at the desired slice
def plot_brain(x, y, z):
    x = np.clip(x, 0,191)
    y = np.clip(y, 0,223)
    z = np.clip(z, 0,191)

    fig, ax = plt.subplots(1,3, figsize=(20,40))
    ax[0].imshow(brain[x, :, :].T, cmap="gray", origin="lower")
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[1].imshow(brain[:, y, :].T, cmap="gray", origin="lower")
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[2].imshow(brain[:, :, z], cmap="gray", origin="lower")
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    return 

interact(plot_brain, x=69, y=83, z=105);


### We need to work with a smaller representation of the data. Let's lower the data quality.

In [ ]:
file_ls = glob.glob('*.nii.gz')
brain_ls = []
for file in file_ls:
  tmp_brain  = nib.load(file).get_fdata()
  brain_ls.append(skimage.measure.block_reduce(tmp_brain, (8,8,8), np.max))

def plot_brain_reduced(n):
  fig, ax = plt.subplots(1,3, figsize=(20,40))
  ax[0].imshow(brain_ls[n][7, :, :].T, cmap="gray", origin="lower")
  ax[0].set_xticks([])
  ax[0].set_yticks([])
  ax[1].imshow(brain_ls[n][:, 10, :].T, cmap="gray", origin="lower")
  ax[1].set_xticks([])
  ax[1].set_yticks([])
  ax[2].imshow(brain_ls[n][:, :, 13], cmap="gray", origin="lower")
  ax[2].set_xticks([])
  ax[2].set_yticks([])

plot_brain_reduced(0)

In [ ]:
print(f"Each brain scan is now an array of: {brain_ls[0].shape}.")
print(f"Or {brain_ls[0].shape[0]*brain_ls[0].shape[1]*brain_ls[0].shape[2]} voxels.")

### I want to find out if I can group them into separate clusters. 
#### What type of learning should I look into? What algorithms can I use?

Some helper functions

In [24]:
from sklearn.decomposition import PCA
brain_np = np.array(brain_ls) 
# We re performing PCA in the list of brains, to try to visualize them in 2 dimensions
pca = PCA(n_components=2)
pca_brain = pca.fit_transform(brain_np.reshape(12, -1))


In [25]:

def plot_cluster_map(centroid0, centroid1, assignements, i):
  # This function generates a plot for us to better visualize how K-Means is performing
  plt.figure()
  plt.title(f"Iteration {i}")
  colour=['red' if i == 0  else 'green' for i in assignements]
  centroid0_pca = pca.transform(centroid0.reshape((1,-1)))
  centroid1_pca = pca.transform(centroid1.reshape((1,-1)))

  plt.scatter(pca_brain[:,0], pca_brain[:,1], c=colour)
  plt.scatter(centroid0_pca[:, 0], centroid0_pca[:, 1],s=200, c='red', marker='X')
  plt.scatter(centroid1_pca[:, 0], centroid1_pca[:, 1],s=200, c='green', marker='X')

  plt.xlabel('1st Pricipal Component')
  plt.ylabel('2nd Pricipal Component')



### Let's try to code by ourselves the K-Means algorithm and apply it to our brain data. 
#### Follow the tips in the comments and remember - google is your friend

In [ ]:

# Flatten data
brain_flatten = brain_np.reshape(12,-1)

# Code a K-Means from scratch
difference = np.random.rand(brain_flatten.shape[1]) * 255
assignments = np.zeros(brain_np.shape[0]) # initializing the assignemnet of the clusters
iteration=0 # just a counting variable


##################### START HERE #####################
# 1. Initialize centroids
# Tip: Check random.choice() to initialize centroids in the place of one of the samples
# centroid0 = ...
# centroid1 = ...

# 2. Repeat until convergence

while(np.all(difference != np.zeros(brain_flatten.shape[1]))): # while the variable difference is not only zeros across every dimension. I.e., the centroid hasn't moved.
    #    2.1 Assign every brain to the closest centroid
    #    Tip: start by measuring the distances to both clusters for each brain. Use np.linalg.norm() for the euclidean distance

    # assignements = ...

    plot_cluster_map(centroid0, centroid1, assignements, iteration)
    #    2.2 Update centroids



    # Here we are updating the difference between iterations to check if the model converged    
    # Do not change
    difference = centroid0 - new_centroid0
    centroid0 = new_centroid0
    centroid1 = new_centroid1
    iteration+=1
    
print("The algorithm has converged!")



## Part 2 - Using Scikit-learn for machine learning

<img src="https://raw.githubusercontent.com/PedroFerreiradaCosta/Lectures/master/tutorial/scikitlearn_homepage.png" alt="FaceFitOpt Logo" width="1000">

https://scikit-learn.org/stable/index.html

### Let's use scikit-learn's K-Means algorithm on our data

In [30]:
# Kmeans on the data - scikit learn
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2, random_state=2)
assignements_sk = model.fit_predict(brain_np.reshape(12, -1))


In [ ]:
centroid0_sk = model.cluster_centers_[0,:]
centroid1_sk = model.cluster_centers_[1,:]

plot_cluster_map(centroid0_sk, centroid1_sk, assignements_sk, -1)

### Are the results different from our implementation? Are the clusters meaningful?
#### Let's look at the data

In [ ]:
# Compare with previous results
plot_cluster_map(centroid0, centroid1, assignements, -1)

## Let's now look into Supervised Learning
### Because we are trying to classify our brains between two categories we will use Logistic regression. 
### What is logistic regression?

<img src="https://raw.githubusercontent.com/PedroFerreiradaCosta/Lectures/master/tutorial/logistic_reg.png" alt="FaceFitOpt Logo" width="500">

<img src="https://raw.githubusercontent.com/PedroFerreiradaCosta/Lectures/master/tutorial/sigmoid.png" alt="FaceFitOpt Logo" width="500">




In [ ]:
# Logistic Regression with labelled outputs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = [0,0,0,1,1,1,0,0,0,0,0,1]
# Flatten data
brain_flatten = brain_np.reshape(12,-1)
# Split data into train and test

# Instantiate logistic regression

# Train the model

# Score


# Plot metrics

## Part 3 - Using Tensorflow to apply deep learning

<img src="https://raw.githubusercontent.com/PedroFerreiradaCosta/Lectures/master/tutorial/tensorflow.png" alt="FaceFitOpt Logo" width="500">


https://www.tensorflow.org/

### We will be creating an autoencoder and will try to reconstruct coronal slides of our brain data

#### Import dependencies

In [39]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

#### Create the model

In [40]:
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])

    self.decoder = tf.keras.Sequential([
      layers.Dense(576, activation='sigmoid'),
      layers.Reshape((24, 24))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    print(encoded)
    decoded = self.decoder(encoded)
    return decoded

#### Initialize the model

In [48]:
latent_dim = 20
autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

#### Let's consider the coronal plane as 2D images for our Auto-encoder.

In [42]:
X = [brain_ls[i][:,j,:] / 255 for i in range(len(brain_ls)) for j in range(brain_ls[i].shape[1])]

# Split data
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=42)

In [43]:
print(f"{len(X_train)} samples will be used for training.")
print(f"{len(X_val)} samples will be used for validation.")
print(f"{len(X_test)} samples will be used for testing.")

214 samples will be used for training.
54 samples will be used for validation.
68 samples will be used for testing.


#### Train the model - why both X_train?

In [ ]:

history = autoencoder.fit(np.array(X_train), np.array(X_train),
                batch_size=10,
                epochs=200,
                shuffle=True,
                validation_data=(np.array(X_val), np.array(X_val)))

#### Let's plot the error progression across training

In [ ]:

plt.plot(range(200), history.history['loss'], label='training loss')
plt.plot(range(200), history.history['val_loss'], label='validation loss')
plt.legend()

In [ ]:
results = autoencoder.evaluate(np.array(X_test), np.array(X_test), batch_size=10)
print(f"Loss in the test set is {results}")

In [52]:
encoded_imgs = autoencoder.encoder(np.array(X_test)).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

#### Let's plot how the autoencoder performed on unseen data (test set)

In [ ]:

def plot_results(sample):
    sample = np.clip(sample, 0, 67)

    fig, ax = plt.subplots(1,2, figsize=(20,40))
    ax[0].imshow(np.array(X_test)[sample, :, :].T, cmap="gray", origin="lower")
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[1].imshow(decoded_imgs[sample, :, :].T, cmap="gray", origin="lower")
    ax[1].set_xticks([])
    ax[1].set_yticks([])

    print("576 values were reduced to these 20 values:")
    print(encoded_imgs[sample, :])
    return 

interact(plot_results, sample=34);
